In [ ]:
import json
import pandas as pd
import numpy as np
import boto3
import getpass

In [33]:
AWS_ACCESS_KEY_ID = getpass.getpass('Enter your AWS Access Key:')

In [ ]:
AWS_SECRET_ACCESS_KEY = getpass.getpass('Enter your AWS Secret Key:')

In [ ]:
AWS_SESSION_TOKEN = getpass.getpass('Enter your AWS Session Token:')

In [ ]:
REGION = getpass.getpass('Enter your region (ex. us-east-1):')

In [6]:
client = boto3.client('textract',
                      aws_access_key_id=AWS_ACCESS_KEY_ID,
                      aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
                      aws_session_token=AWS_SESSION_TOKEN,
                      region_name=REGION)
s3 = boto3.resource('s3',
                    aws_access_key_id=AWS_ACCESS_KEY_ID,
                    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
                    aws_session_token=AWS_SESSION_TOKEN,
                    region_name=REGION)

bucket = s3.Bucket('project-350-warner')

In [7]:
obj_list = [i.key for i in bucket.objects.all()]
# woz_list = [i for i in obj_list if 'pWOZ' in i]
# london_list = [i for i in obj_list if 'pLND' in i]
# phrase_list = [i for i in obj_list if 'pPHR' in i]

sample_list = [i for i in obj_list if i.endswith('.png') and i.startswith('AllHandwriting')]

In [8]:
def get_n_samples(n,use_all=False):
    global sample_list
    if use_all:
        n = len(sample_list)
    file_dict = {}
    count = 0
    for i in sample_list[:n]:
        resp = client.detect_document_text(Document={'S3Object':
                                                    {'Bucket': 'project-350-warner','Name': f'{i}'}})
        file_dict.update({i.split('/')[-1][:-4]:resp})
        count += 1
        print(f'Finished file {count}/{n}')
    return file_dict

In [9]:
# Generate a dictionary containing n samples of responses
# In our testing, we ran all samples through Textract with get_n_samples([any n], use_all=True)
responses = get_n_samples(5)

Finished file 1/5
Finished file 2/5
Finished file 3/5
Finished file 4/5
Finished file 5/5


In [11]:
# Taking a look at the output of one of the sample responses.
key1 = list(responses.keys())[0]
responses.get(key1)

{'DocumentMetadata': {'Pages': 1},
 'Blocks': [{'BlockType': 'PAGE',
   'Geometry': {'BoundingBox': {'Width': 0.9998016953468323,
     'Height': 1.0,
     'Left': 0.0,
     'Top': 0.0},
    'Polygon': [{'X': 0.0, 'Y': 0.0},
     {'X': 0.9998016953468323, 'Y': 1.0994781469914397e-16},
     {'X': 0.9998016953468323, 'Y': 1.0},
     {'X': 0.0, 'Y': 1.0}]},
   'Id': '5dba2d53-f143-4782-80ff-3bad5df77c4e',
   'Relationships': [{'Type': 'CHILD',
     'Ids': ['3b6af0c8-4922-43cd-b72b-305162861685',
      '5566daf8-365d-41e6-be62-b678e729c586',
      '6e7f7ccb-90e8-4974-acf0-16e4bb77a93f',
      '8c9784a9-2915-4aec-a494-3dd6a90e006a',
      '60001886-6268-4587-9fca-efd3b5184ebf',
      '1163cf5b-9a13-4b58-b397-8f4bfc000948',
      'fa5c261e-dbac-4a68-b664-37b149964291',
      'bdb912ef-16ae-4059-bca3-4f2c40651d4c',
      '638203ad-ea85-40c3-b09c-74088f460fdc',
      '4ee7a856-c72d-46da-b0b2-f1e242bcdeca',
      'b5f03c75-ddf6-4179-8e54-fc2a2cca86aa']}]},
  {'BlockType': 'LINE',
   'Confidence'

In [12]:
# Save these responses to a JSON file for cleaning in another notebook.
with open("sample_responses.json", "w") as outfile:
    json.dump(responses, outfile)

In [15]:
# To reproduce exactly what is in the repo, run the next few blocks
# However, this will take hours to generate all responses.
all_responses = get_n_samples(1, use_all=True)

split_idx = len(all_responses.keys())//2
d1 = dict(list(all_responses.items())[:split_idx])
d2 = dict(list(all_responses.items())[split_idx:])

Finished file 1/3754
Finished file 2/3754
Finished file 3/3754
Finished file 4/3754
Finished file 5/3754
Finished file 6/3754
Finished file 7/3754
Finished file 8/3754
Finished file 9/3754
Finished file 10/3754
Finished file 11/3754
Finished file 12/3754
Finished file 13/3754
Finished file 14/3754
Finished file 15/3754
Finished file 16/3754
Finished file 17/3754
Finished file 18/3754


KeyboardInterrupt: 

In [ ]:
with open("new_responses1.json", "w") as outfile:
    json.dump(d1, outfile)

with open("new_responses2.json", "w") as outfile:
    json.dump(d2, outfile)